In [3]:
import pandas as pd
import numpy as np
import json
import re
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D  # for 3D plotting
import plotly.express as px
from sklearn.manifold import TSNE
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
import plotly.io as pio
import requests
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
import os
from os import getenv
from dotenv import load_dotenv
import time

load_dotenv("../.env",override=True)
GEO_KEY = getenv("GOOGLE_API_KEY")

x_chat = chatGPT()
x_gemini = gemini()


In [11]:
df = pd.read_csv("../output/bls_df.csv",index_col=0)
# tsne = pd.read_csv("../output/tsne.csv",index_col=0)
# df = pd.concat([df,tsne],axis=1)

# df.example_task_embedding = df.example_task_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
# df.onet_task_embedding = df.onet_task_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
# df.onet_title_embedding = df.onet_title_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])



In [13]:


wage_employment = pd.read_csv("../input/wage_employment_2022/national2022.csv")
wage_employment = wage_employment[wage_employment.O_GROUP == "detailed"]
wage_employment = wage_employment.rename({"OCC_CODE": "Detailed Occupation"}, axis=1)
wage_employment = wage_employment.replace("#", np.nan).replace("*", np.nan).replace({",": ""}, regex=True)
wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP","JOBS_1000"]] = wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP","JOBS_1000"]].astype("float")
wage_employment["H_MEDIAN"] = wage_employment["H_MEDIAN"].round(0)
wage_employment = wage_employment[["Detailed Occupation","H_MEDIAN","A_MEDIAN","TOT_EMP","JOBS_1000"]]
wage_employment.columns = ["Detailed Occupation","H_MEDIAN_US","A_MEDIAN_US","TOT_EMP_US","JOBS_1000_US"]


all_tasks = df.groupby("Task").aggregate({"onet_weight":"sum"}).reset_index()
task_statements = pd.read_csv("../input/onet/Task Statements.csv")[["Task","Title"]]
all_tasks = all_tasks.merge(task_statements, on=["Task"], how="outer")
onet_occ = pd.read_csv("../input/onet/Occupation Data.csv")[["O*NET-SOC Code","Title"]]
onet_occ.columns = ["Detailed Occupation","onet_title"]
onet_occ["Detailed Occupation"] = onet_occ["Detailed Occupation"].apply(lambda x: x[:-3])
all_tasks = all_tasks.merge(onet_occ, left_on="Title",right_on="onet_title").drop(columns=["Title"])

all_tasks = all_tasks.replace(np.nan,0)
perc_of_10K = (wage_employment.groupby("Detailed Occupation")["TOT_EMP_US"].first()/wage_employment["TOT_EMP_US"].sum() * 10000).reset_index() # num of people for every 10K workers
perc_of_10K = perc_of_10K.rename({"TOT_EMP_US":"Percent of 10K workers"},axis=1)


all_tasks = all_tasks.merge(perc_of_10K, on="Detailed Occupation")
ratios = all_tasks['onet_weight'] / all_tasks['Percent of 10K workers']
all_tasks["automated_weight"] = np.minimum(ratios, 1)


all_occupations = all_tasks.groupby("Detailed Occupation").aggregate({"automated_weight":"sum","Task":"count", "onet_title":"first"})
all_occupations.columns = ["occupation_# tasks automated","occupation_# of tasks","onet_title"]
all_occupations["occupation_onet_rating"] = all_occupations["occupation_# tasks automated"]/all_occupations["occupation_# of tasks"]
df = df.merge(all_tasks[["Task","automated_weight","Percent of 10K workers"]], on="Task")
df = df.merge(all_occupations,on=["Detailed Occupation","onet_title"])

df = df.merge(wage_employment, on="Detailed Occupation")
all_occupations = all_occupations.merge(wage_employment, on="Detailed Occupation")
all_tasks = all_tasks.merge(wage_employment, on="Detailed Occupation")
all_tasks = all_tasks.rename({"Title":"onet_title"},axis=1)

In [14]:
pd.read_csv("../input/onet/Skills.csv")

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
0,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,IM,Importance,4.12,8.0,0.13,3.88,4.37,N,NaN,07/2014,Analyst
1,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,LV,Level,4.75,8.0,0.16,4.43,5.07,N,N,07/2014,Analyst
2,11-1011.00,Chief Executives,2.A.1.b,Active Listening,IM,Importance,4.12,8.0,0.13,3.88,4.37,N,NaN,07/2014,Analyst
3,11-1011.00,Chief Executives,2.A.1.b,Active Listening,LV,Level,4.88,8.0,0.23,4.43,5.32,N,N,07/2014,Analyst
4,11-1011.00,Chief Executives,2.A.1.c,Writing,IM,Importance,4.00,8.0,0.00,4.00,4.00,N,NaN,07/2014,Analyst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61105,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.b,Management of Financial Resources,LV,Level,1.12,8.0,0.13,0.88,1.37,N,N,08/2019,Analyst
61106,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.c,Management of Material Resources,IM,Importance,2.00,8.0,0.00,2.00,2.00,N,NaN,08/2019,Analyst
61107,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.c,Management of Material Resources,LV,Level,1.88,8.0,0.13,1.63,2.12,N,N,08/2019,Analyst
61108,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.d,Management of Personnel Resources,IM,Importance,2.88,8.0,0.13,2.63,3.12,N,NaN,08/2019,Analyst


In [15]:
dwa = pd.read_csv("../input/onet/Tasks to DWAs.csv")[3:].reset_index(drop=True)
dwa = dwa[["DWA ID","DWA Title","Task"]]
# dwa = dwa.rename({"ID":"Task ID"},axis=1)
grouped = dwa.groupby("Task").aggregate({"DWA ID":"count"}).apply(lambda x: 1/x).reset_index()
grouped = grouped.rename({"DWA ID":"dwa_count"},axis=1)
dwa = dwa.merge(grouped,on="Task")
dwa_ref = pd.read_csv("../input/onet/DWA Reference.csv")[["Element Name","DWA ID"]]
dwa_ref = dwa_ref.rename({"Element Name":"activity"},axis=1)

tf = df.merge(dwa, on="Task",how="left")
tf = tf.merge(dwa_ref, on="DWA ID",how="left")
print(len(tf[tf["DWA ID"].isnull()]))

print(len(tf))

0
234


In [16]:
# skill_activity = pd.read_csv("../input/onet/Skills to Work Activities.csv")[["Work Activities Element Name", "Skills Element Name"]]
# skill_activity.columns = ["activity", "skill"]
# skill_activity["skill_activity"] = skill_activity["skill"]+" "+skill_activity["activity"]
# skill_activity = x_chat.run_batch_embeddings(skill_activity, "activity")
# skill_activity = x_chat.run_batch_embeddings(skill_activity, "skill")
# skill_activity = x_chat.run_batch_embeddings(skill_activity, "skill_activity")
# skill_activity.to_csv("../input/onet/activity_skill_embeddings.csv", index=False)



In [17]:
def parse_bls(wage_employment,type):
    wage_employment = wage_employment[wage_employment.O_GROUP == "detailed"]
    wage_employment = wage_employment.rename({"OCC_CODE": "Detailed Occupation"}, axis=1)
    wage_employment = wage_employment.replace("#", np.nan).replace(r"\*+", np.nan, regex=True).replace(",", "", regex=True)
    wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP","JOBS_1000"]] = wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP", "JOBS_1000"]].astype("float")
    wage_employment["H_MEDIAN"] = wage_employment["H_MEDIAN"].round(0)
    wage_employment = wage_employment[["AREA","Detailed Occupation","H_MEDIAN","A_MEDIAN","TOT_EMP","JOBS_1000"]]
    wage_employment.columns = ["AREA","Detailed Occupation","H_MEDIAN_"+type,"A_MEDIAN_"+type,"TOT_EMP_"+type, 'JOBS_1000_'+type]
    return wage_employment


nonmetro = pd.read_csv("../input/wage_employment_2022/rural2022.csv")
metro = pd.read_csv("../input/wage_employment_2022/metro2022.csv")
nonmetro = parse_bls(nonmetro,"RURAL")
metro = parse_bls(metro,"METRO")

#figures out how much to weight each area
def calculate_weights(df,type):
    grouped = df.groupby("Detailed Occupation")["TOT_EMP_"+type].sum().reset_index()
    grouped = grouped.rename({"TOT_EMP_"+type: "sum_"+type}, axis=1)
    df = df.merge(grouped, on="Detailed Occupation")
    df["weight_area_"+type] = df["TOT_EMP_"+type] / df["sum_"+type]
    df["weighted_JOBS_1000_"+type] = df["JOBS_1000_"+type]*df["weight_area_"+type]
    return df
metro = calculate_weights(metro,"METRO")
nonmetro = calculate_weights(nonmetro,"RURAL")

metro_grouped = metro.groupby("Detailed Occupation").aggregate({"H_MEDIAN_METRO":"mean","TOT_EMP_METRO":"sum","A_MEDIAN_METRO":"mean", "weighted_JOBS_1000_METRO":"sum"},axis=1).reset_index()
nonmetro_grouped = nonmetro.groupby("Detailed Occupation").aggregate({"H_MEDIAN_RURAL":"mean","TOT_EMP_RURAL":"sum","A_MEDIAN_RURAL":"mean","weighted_JOBS_1000_RURAL":"sum"},axis=1).reset_index()

df = df.merge(metro_grouped,on="Detailed Occupation",how="left")
df = df.merge(nonmetro_grouped,on="Detailed Occupation",how="left")

In [20]:
metro.columns = [col.replace("_METRO","") for col in metro.columns]
metro["Type"] = "METRO"
nonmetro.columns = [col.replace("_RURAL","") for col in nonmetro.columns]
nonmetro["Type"] = "RURAL"
locations = pd.concat([metro, nonmetro],axis=0)
locations = locations.merge(all_occupations,on="Detailed Occupation")
locations["weighted_onet_rating"] = locations["JOBS_1000"]*locations["occupation_onet_rating"]
locations = locations.groupby("AREA").aggregate({"weighted_onet_rating":"sum", "Type":"first"}).reset_index()
area_code = pd.read_csv("../output/geography.csv")[["AREA", "Latitude","Longitude","Boundary"]]
area_code.columns = ["state","place","AREA","name"]
locations = locations.merge(area_code, on="AREA",how="left")
locations = locations.dropna(subset=["name"])
locations


FileNotFoundError: [Errno 2] No such file or directory: '../input/wage_employment_2022/area_definitions_m2022.csv'

In [5]:
# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Load area_code dataframe
area_code = pd.read_csv("../input/wage_employment_2022/area_definitions_m2022.csv")[[ 'State abbreviation','May 2022 MSA name','May 2022 MSA code ','County name (or Township name for the New England states)']]
area_code.columns = ["state", "place", "AREA", "name"]
area_code["Latitude"] = "N/A"
area_code["Longitude"] = "N/A"
area_code["Boundary"] = "N/A"

async def get_lat_long(session, msa_name):
    while True:
        try:
            url = f"https://maps.googleapis.com/maps/api/geocode/json?address={msa_name}&key={GEO_KEY}"
            async with session.get(url) as response:
                while True:
                    data = await response.json()
                    if data['status'] == 'OK':
                        bounds = data['results'][0]['geometry']
                        location = data['results'][0]['geometry']['location']
                        return location['lat'], location['lng'], bounds
                    else:
                        if data['status'] != "OVER_QUERY_LIMIT":
                            return None, None, None
                        print(f"Geocoding error for {msa_name}: {data['status']}")
                        await asyncio.sleep(2)
        except Exception as e:
            print(f"Error geocoding {msa_name}: {e}")
            return None, None, None

# Define the main asynchronous function
async def process_area_code_batch(area_code_batch):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for index, row in area_code_batch.iterrows():
            msa_name = row['name'] + " " + row["state"]
            tasks.append(get_lat_long(session, msa_name))
        
        results = await asyncio.gather(*tasks)

        for (index, (lat, lng, bounds)) in zip(area_code_batch.index, results):
            area_code_batch.at[index, 'Latitude'] = lat
            area_code_batch.at[index, 'Longitude'] = lng
            area_code_batch.at[index, 'Boundary'] = bounds

    return area_code_batch

# Function to process the entire DataFrame in batches
async def process_area_code_in_batches(area_code, batch_size=100):
    batches = [area_code.iloc[i:i + batch_size] for i in range(0, len(area_code), batch_size)]
    results = []
    for i, batch in enumerate(batches):
        print(f"Processing batch {i + 1}/{len(batches)}")
        result_batch = await process_area_code_batch(batch)
        results.append(result_batch)
        await asyncio.sleep(10)
    return pd.concat(results)


area_code = await process_area_code_in_batches(area_code, batch_size=100)






Processing batch 1/50
Processing batch 2/50
Processing batch 3/50
Processing batch 4/50
Processing batch 5/50
Processing batch 6/50
Processing batch 7/50
Processing batch 8/50
Processing batch 9/50
Processing batch 10/50
Processing batch 11/50
Processing batch 12/50
Processing batch 13/50
Processing batch 14/50
Processing batch 15/50
Processing batch 16/50
Processing batch 17/50
Processing batch 18/50
Processing batch 19/50
Processing batch 20/50
Processing batch 21/50
Processing batch 22/50
Processing batch 23/50
Processing batch 24/50
Processing batch 25/50
Processing batch 26/50
Processing batch 27/50
Processing batch 28/50
Processing batch 29/50
Processing batch 30/50
Processing batch 31/50
Processing batch 32/50
Processing batch 33/50
Processing batch 34/50
Processing batch 35/50
Processing batch 36/50
Processing batch 37/50
Processing batch 38/50
Processing batch 39/50
Processing batch 40/50
Processing batch 41/50
Processing batch 42/50
Processing batch 43/50
Processing batch 44/

In [31]:
area_code

,state,place,AREA,name
0,AL,"Montgomery, AL",33860,Autauga County
1,AL,"Daphne-Fairhope-Foley, AL",19300,Baldwin County
2,AL,Southeast Alabama nonmetropolitan area,100004,Barbour County
3,AL,"Birmingham-Hoover, AL",13820,Bibb County
4,AL,"Birmingham-Hoover, AL",13820,Blount County
...,...,...,...,...
4932,PR,"San Juan-Carolina-Caguas, PR",41980,Vega Baja Municipio
4933,PR,Puerto Rico nonmetropolitan area,7200006,Vieques
4934,PR,"Ponce, PR",38660,Villalba Municipio
4935,PR,"San Juan-Carolina-Caguas, PR",41980,Yabucoa Municipio
